# Quantum Cryptography using BB84
This third exercise focuses on **[BB84](https://en.wikipedia.org/wiki/BB84)**, a cryptography protocol developed in 1984 by one of the most famous IBMers, Charles Bennett, together with his colleague Gilles Brassard. This scheme was realized five years later in the first demonstration of [quantum key distribution](https://en.wikipedia.org/wiki/Quantum_key_distribution) by Bennett and colleague John Smolin at IBM [C. H. Bennett, F. Bessette, G. Brassard, L. Salvail, and J. Smolin, J. of Cryptography **5**, 3 (1992) ].  Both Charles and John are still members of the IBM Quantum team.



<img src="https://raw.githubusercontent.com/qiskit-community/may4_challenge_exercises/master/ex03/charlie_john_qkd.jpg" width="500"/>
<center>Charles Bennett and John Smolin working on the first demonstration of quantum key distribution, IBM T.J. Watson Research Center.</center>



The goal of the BB84 protocol is to create a secret key between two parties, Alice and Bob, that can then be used by both parties to encrypt and decrypt a hidden message. In this exercise we will guide you through the different steps of the protocol to create such a secret key to decrypt our encrypted message.


## BB84 protocol

Let's walk through the steps of the BB84 protocol:

1. In the first step, Alice chooses two random bit strings, $k$ and $b$, that each consist of $n$ bits. Her bit string $k$ contains the actual bits she wants to encode (out of which the key will later be formed), while $b$ determines the bases in which she will encode her bits. For $b_i=0$ (i.e., if the $i^{th}$ bit is zero), she encodes the $i^{th}$ qubit in the standard $\{|0\rangle, |1\rangle \}$ basis, while for $b_i=1$, she encodes it in the $\{|+\rangle, |-\rangle \}$ basis, where $|+\rangle:=\frac{1}{\sqrt{2}}(|0\rangle +|1\rangle)$, $|-\rangle:=\frac{1}{\sqrt{2}}(|0\rangle -|1\rangle)$. 
This becomes more illustrative when representing each basis by two perpendicular arrows, where the two different bases are rotated by $45^\circ$.
The encoding of each qubit $q_i$ would therefore look like the following:

<img src="https://raw.githubusercontent.com/qiskit-community/may4_challenge_exercises/master/ex03/encoding_Alice.png" alt="drawing" width="300"/>

2. After encoding her $n$ qubits, Alice sends these qubits to Bob. Bob also chooses a random bit string $\tilde{b}$ consisting of $n$ bits that determines in which bases he is going to perform measurements. He stores the outcomes of his measurements $\tilde{k_i}$ together with the corresponding basis bits $\tilde{b_i}$ in a table.

3. Next, Alice and Bob compare their basis bits $b_i$ and $\tilde{b}_i$. Whenever $b_i \neq \tilde{b}_i$, Bob measured in a different basis than Alice's qubit was encoded in, so he gets each outcome with probability $\frac{1}{2}$. Alice and Bob therefore discard all key bits corresponding to these basis bits. If $b_i = \tilde{b}_i$, however, they prepared and measured the qubit in the same basis, so (unless someone eavesdropped) Bob will get the key bit that Alice encoded, $\tilde{k}_i = k_i$. These outcomes then compose the key.

## An illustrated example

Suppose Alice's random bit strings are $k=`0111001`$ and $b=`1101000`$ and Bob's random bit string is $\tilde{b}=`1001101`$. Try to understand the other entries in the table below. Note that in the case where the basis bits are different, Bob has a 50% chance to get each outcome, so here one of them was chosen randomly.

<img src="https://raw.githubusercontent.com/qiskit-community/may4_challenge_exercises/master/ex03/example_bb84.png" alt="drawing" width="600"/>

The key produced in this example would be '0110'. To make sure that the key is secret and correct, Alice and Bob would "sacrifice" some of their key bits to check that no one eavesdropped. If someone had measured a qubit on the way, this could have changed the state of that qubit and with probability $\frac{1}{4}$, Bob's and Alice's key bits will be different. By checking $m$ bits, the probability to not notice an eavesdropper decreases as $\left(\frac{3}{4}\right)^m$. Thus, if they check enough bits and they are all the same, they can assume that no one eavesdropped and their key is secret. However, to keep things simple, we will not perfom these tests in this excercise. Instead, all bits of the key will be used.

### Message encrpytion
Once a secret key is distributed, Alice can encrypt her message by using the so-called [one-time pad](https://en.wikipedia.org/wiki/One-time_pad) technique: she simply adds the key bits on top of her secret message bits that she wants to send. Using the example above, her key is $\text{key}=`0110`$. If her secret message bit string is $m=`1100`$, the encrypted message will be $c=m\oplus \text{key} \mod 2 = `1010`$. Bob can then decrypt the message by adding his key on that encrypted message, $m=c\oplus \text{key} \mod 2$.

## Your task 

In the following tasks, you play the role of Bob. You will create such a secret key with Alice and use it to decrypt the encrypted message from her.

(**2 points**)


In [ ]:
%matplotlib inline

# Importing standard Qiskit libraries
import random
from qiskit import *
from qiskit.tools.jupyter import *
from qiskit.visualization import *

#Get the library to check the answers
%pip install -I git+https://github.com/mnp-club/MnP_QC_Workshop.git
from mnp_qc_workshop_2020.bb84 import *

# Configuring account
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')  # with this simulator it wouldn't work \

# Initial setup
random.seed(64) # do not change this seed, otherwise you will get a different key



### Simulating this protocol in Qiskit

(**8 points**)

In this exercise, there are three steps. Each of these three steps is completed $n=16$ times. Before the protocol begins, Alice will choose two random bit strings, $k$ and $b$.

1. Alice will prepare each qubit using the function `Alice_prepare_qubit`, which is already written for you.

2. Bob measures his qubit using a specific set of bases, which we have given you in a variable called `bases`. **You will supply the procedure that Bob takes in the function `Bob_measure_qubit`.**

3. A quantum circuit for this sequence of operations is created. It will be called `this_qubit_circuit` for each qubit, and all such circuits are collected together in an array called `all_qubit_circuits`. We have supplied the code to do this.

Finally, we run `all_qubit_circuits` on the IBM high-performance cloud simulator called `ibmq_qasm_simulator` and collect the results of the measurements into a bitstring called `bits`. We have supplied the code to do this.

You will then follow the protocol to decrypt a message using the extracted key from the BB84 protocol.


### i) Execute step 2 of the BB84 protocol to get your bitstring
To do so, you need a random bit string $\tilde{b}$ that determines the bases in which you should perform the measurements. In order to reproduce the key that we used to encrypt our secret message, we provide you with this "random" `bases` bit string of 16 bits that uses seed "64".
**Perform measurements in the bases corresponding to the given bit string and return the outcome as a bit string in the form '$\tilde{b}_0\tilde{b}_1...\tilde{b}_{n-1}$'.** Note that Qiskit returns outcomes always in reverse order, so if $|\tilde{q}_0\rangle = |0 \rangle $ and $|\tilde{q}_1\rangle = |\tilde{q}_2\rangle = |1 \rangle $, it will return the outcome '110'. You can check whether your bit string is correct by `check_bits(bitstring)`.

In [ ]:
# This is your 'random' bit string that determines your bases
numqubits = 16
bob_bases = str('{0:016b}'.format(random.getrandbits(numqubits)))

def bb84():
    print('Bob\'s bases:', bob_bases)

    # Now Alice will send her bits one by one...
    all_qubit_circuits = []
    for qubit_index in range(numqubits):

        # This is Alice creating the qubit
        thisqubit_circuit = alice_prepare_qubit(qubit_index)

        # This is Bob finishing the protocol below
        bob_measure_qubit(bob_bases, qubit_index, thisqubit_circuit)
        
        # We collect all these circuits and put them in an array
        all_qubit_circuits.append(thisqubit_circuit)
        
        
    # Now execute all the circuits for each qubit
    results = execute(all_qubit_circuits, backend=backend, shots=1).result()
    # And combine the results
    bits = ''
    for qubit_index in range(numqubits):
        bits += [measurement for measurement in results.get_counts(qubit_index)][0]
        
    return bits

# Here is your task
def bob_measure_qubit(bob_bases, qubit_index, qubit_circuit):
    #
    #
    #Your code here
    #
    #
    
bits = bb84()      
print('Bob\'s bits: ', bits)
check_bits(bits)

After you performed your measurements, Alice announces her bases bits which are stored in <code>alice_bases</code> in the next cell

### ii) Execute step 3 of the BB84 protocol to extract the key
To do so, compare Alice's bases bits $b$ to your bases bits $\tilde{b}$ (given in the previous step) and extract the key by keeping only the outcomes when your bases were the same.
You can check whether your key is correct by `check_key(key)`.

In [ ]:
alice_bases = '0100000101011100' # Alice's bases bits
key = ''
#
#
#Your code here
#
#
print(key)
check_key(key)

Using this key, Alice can now send you her private encrypted message. While for full security a key would only be used once, she will now use her 8-bit-long key 43 times in a row to encrypt her 344-bit-long message. Her encrypted message is stored in <code>message</code> variable of the next cell.

A small note I would like to make is that generally the key is kept to be of a larger length since this ratio is actually very skewed but for demonstration purposes of the concept in play we will use it in this way.

### iii) Decrypt the message using the extracted key
The message encryption section in the introduction describes how to do so. You can check whether your decrypted message bit string is correct by <code>check_decrypted(decrypted)</code>.

In [ ]:
message = get_message()# encrypted message
#
#
#Your code here
#
#
print(decrypted)
check_decrypted(decrypted)

Now you would have obtained the decrypted message. You have a pleasent surprise waiting for you after you convert this using ASCII notation. In the cell below, convert every of the forty three 8 bit pieces of the string to a character using the regular ASCII notation. The <code>check_message</code> function will tell you whether you have correctly converted it or not.

In [ ]:
decrypted_to_string_ASCII = ''
#
#
#Your code here
#
#
check_message(decrypted_to_string_ASCII)

The documentation in this notebook has been taken mainly from <a href="https://github.com/qiskit-community/may4_challenge_exercises/blob/master/ex03/Challenge3_BB84.ipynb">here</a>. Kudos to the qiskit team for documenting this so well.